In [69]:
import re, string
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import *
from pyspark.sql.functions import *
from pyspark.ml import Pipeline

In [70]:
from sklearn.datasets import fetch_20newsgroups

In [71]:
#conf = SparkConf().setAppName("TP3 - BD2")
sc = SparkSession.builder.appName("tp3").config("spark.logConf", "true").getOrCreate()

In [72]:
dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Funcoes

https://spark.apache.org/docs/latest/ml-features

In [73]:
def clean_dataset(dataset):
    for data in dataset:
        if data[0] == '\n':
            data[0] = data[0].replace('\n','') 
    return dataset

In [84]:
#https://stackoverflow.com/questions/46975929/how-can-i-calculate-the-jaccard-similarity-of-two-lists-containing-strings-in-py
def jacDistance(set1, set2):
    s1 = set(set1)
    s2 = set(set2)
    return len(s1.intersection(s2)) / float(len(s1.union(s2)))

# Context
This dataset is a collection newsgroup documents. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

In [74]:
newsgroup_aux = list(zip(dataset.data, dataset.target.tolist()))

In [75]:
newsgroup_aux[:] = [x for x in newsgroup_aux if (len(x[0].replace('\n', '')) > 0)]

In [76]:
df = sc.createDataFrame(newsgroup_aux, schema=['raw_data', 'real_value'])

In [77]:
df = df.withColumn('id', monotonically_increasing_id())

In [78]:
tokenizer = Tokenizer(inputCol = 'raw_data', outputCol = 'words')
#wordsData = tokenizer.transform(sentenceData)

In [79]:
vectorizer = CountVectorizer(inputCol = 'words', outputCol='features', vocabSize=3)

In [80]:
hashLSH = MinHashLSH(inputCol='features', outputCol='hashes', numHashTables=5)

In [81]:
pipeline = Pipeline(stages = [tokenizer, vectorizer, hashLSH])

In [82]:
post_process = pipeline.fit(dataset = df).transform(df)

In [83]:
post_process.select('features').show(10)

+--------------------+
|            features|
+--------------------+
|(3,[0,1,2],[12.0,...|
|(3,[0,1],[19.0,1.0])|
|(3,[0,1,2],[194.0...|
|(3,[0,1,2],[5.0,1...|
|(3,[0,1,2],[14.0,...|
| (3,[0,1],[6.0,1.0])|
|       (3,[0],[3.0])|
|(3,[0,1,2],[11.0,...|
|(3,[0,1,2],[7.0,3...|
|(3,[0,1,2],[11.0,...|
+--------------------+
only showing top 10 rows



## Estrutura do df:
id: bigint

raw_data: string

real_value: bigint

words: array<string>
    
features: vector
    
hashes: array<vector>

A ideia aqui de usar a distancia de Jaccard eh tentar descobrir a similaridade entre dois sets e poupar o nosso KNN de amostras muito diferentes

In [86]:
selected_data = post_process.select('id', 'real_value', 'features', 'hashes')

In [87]:
for data in selected_data:
    data_dict = {}
    data_dict['id'] = data[id]

AttributeError: 'builtin_function_or_method' object has no attribute '_get_object_id'